In [30]:
%%capture
!pip install langchain==0.1.4 openai==1.10.0 langchain-openai

In [31]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

# Prompt Pipelining Overview


- 🧱 **Modularity**: Mix and match prompt pieces like LEGO blocks.
- 👓 **Readability**: Break down complex prompts into easy bits.
- 🧠**Flexibility**: Craft prompts on-the-go with logic-based assembly.
- 🔄 **Efficiency**: Loop to append for scenarios like few-shot learning.
- 📝✨**Hybrid Construction**: Combine fixed text with variable-filled templates for structure and spontaneity.
- 💬 **Chat-Friendly**: Create conversational prompts by stacking messages.
- 🛠️ **Customizability**: Let users build prompts with their own components.

**String Prompt Pipelining:**

- 🔗 **Sequential Flow**: Link templates or strings in order, starting with a prompt.

Prompt pipelining turns the art of prompt crafting into a modular, efficient process, perfect for those looking to streamline and enhance their prompt design. 💡


In [32]:
from langchain.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template("I'm heading to {destination}. ")
    + "Recommend a great {activity} spot!")

prompt

PromptTemplate(input_variables=['activity', 'destination'], template="I'm heading to {destination}. Recommend a great {activity} spot!")

In [33]:
prompt = prompt + "\n\nAlso, any local delicacies I should try?"
prompt

PromptTemplate(input_variables=['activity', 'destination'], template="I'm heading to {destination}. Recommend a great {activity} spot!\n\nAlso, any local delicacies I should try?")

In [34]:
print(prompt.template)

I'm heading to {destination}. Recommend a great {activity} spot!

Also, any local delicacies I should try?


In [35]:
prompt.format(destination="Punjab", activity="dining")

"I'm heading to Punjab. Recommend a great dining spot!\n\nAlso, any local delicacies I should try?"

### The key differences between prompt pipelining and multi-input prompts are:

🚂 **Composition**: Pipelining links individual prompts into a cohesive journey.

🖼️ **Independence**: Each pipeline component is crafted separately before integration.

🚶‍♂️ **Sequence**: Pipelining lines up components, unlike multi-input prompts that handle inputs collectively.

📋 **Reuse**: Pipelining excels in reusing pieces; multi-input prompts manage multiple data points in one go.

📖 **Outcome**: Pipelining produces a single narrative; multi-input prompts generate a combined result.

🧱 **Construction**: Pipelining is about assembling prompts step by step, while multi-input prompts are about managing various inputs at once.

In short, pipelining is like creating a melody note by note, whereas multi-input prompts are like playing chords, hitting multiple notes simultaneously.


# Use in a chain

In [36]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [37]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.75)

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

for chunk in chain.stream({"destination":"Punjab", "activity":"dining"}):
  print(chunk, end="", flush=True)

One highly recommended dining spot in Punjab is the Bukhara Grill, known for its delicious North Indian cuisine and traditional tandoori dishes. For local delicacies, be sure to try the famous Punjabi dish called Butter Chicken, as well as Chole Bhature, a spicy chickpea curry served with fried bread. Another must-try is Makki di Roti, a cornmeal flatbread, paired with Sarson da Saag, a mustard greens curry. These dishes will give you a true taste of the Punjab region's rich and flavorful cuisine. Enjoy your trip!

In [38]:
prompt = prompt + " How should I greet the locals in a jolly, informal manner?"


In [39]:
prompt.template

"I'm heading to {destination}. Recommend a great {activity} spot!\n\nAlso, any local delicacies I should try? How should I greet the locals in a jolly, informal manner?"

In [40]:
chain = prompt | llm | output_parser

for chunk in chain.stream({"destination":"Punjab", "activity":"dining"}):
  print(chunk, end="", flush=True)

For a great dining spot in Punjab, I would recommend trying Beera Chicken House in Amritsar. They are famous for their delicious tandoori chicken and other Punjabi dishes.

Some local delicacies you should definitely try are butter chicken, sarson da saag with makki di roti, chole bhature, and Amritsari kulcha.

To greet the locals in a jolly, informal manner, you can simply say "Sat Sri Akal" with a warm smile. This is a common Punjabi greeting that will surely be appreciated by the locals.

# Example usecase

In [41]:
class TravelChatbot:
    def __init__(self, base_template):
        self.model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.75)
        self.base_prompt = PromptTemplate.from_template(base_template)

    def append_to_prompt(self, additional_text):
        self.base_prompt += additional_text

    def run_chain(self, destination, activity):
        output_parser = StrOutputParser()
        chain = self.base_prompt | self.model | output_parser
        for chunk in chain.stream({"destination":destination, "activity":activity}):
          print(chunk, end="", flush=True)

# Usage
base_template = "I'm heading to {destination}. Recommend a great {activity} spot!"

chatbot = TravelChatbot(base_template)

# Basic prompt
chatbot.run_chain(destination="Punjab", activity="dining")

If you're heading to Punjab, I highly recommend trying out the famous Punjabi Dhaba for an authentic dining experience. These roadside eateries serve delicious, traditional Punjabi cuisine, including dishes like tandoori chicken, butter chicken, and various types of bread like naan and paratha. You'll also get to enjoy the vibrant and lively atmosphere of these dhabas, making it a memorable dining experience. Enjoy your trip to Punjab!

In [42]:
# Append more to the prompt and run again
chatbot.append_to_prompt("\n\nAlso, any local delicacies I should try?")

chatbot.run_chain(destination="Punjab", activity="dining")

One great dining spot in Punjab is Kesar Da Dhaba in Amritsar. It is known for its delicious traditional Punjabi cuisine and has been a favorite among locals and tourists for many years.

When in Punjab, you must try the popular local delicacies such as Makki di Roti and Sarson da Saag, Chole Bhature, Tandoori Chicken, and Amritsari Kulcha. Don't forget to also try the famous Punjabi sweets like Jalebi, Gulab Jamun, and Lassi. Enjoy your culinary experience in Punjab!

In [43]:
chatbot.append_to_prompt(" How should I greet the locals in a friendly, informal, jolly colloquial manner?")

chatbot.run_chain(destination="Punjab", activity="dining")

One great dining spot in Punjab is Barbeque Nation in Ludhiana, known for its delicious barbecue and grill options. 

Some local delicacies you should try in Punjab include tandoori chicken, butter chicken, makki di roti and sarson da saag, and lassi.

To greet the locals in a friendly, informal, jolly colloquial manner, you can use the Punjabi greeting "Sat Sri Akal" which means "Blessings to you." It's a warm and respectful way to greet people in Punjab.

# Chat Prompt Pipeline

🧩 **Composition**: Chat prompt pipelining turns reusable message blocks into a complete conversation flow.

🛠️ **Versatility**: Mix and match static messages with dynamic templates for a custom dialogue.

🔗 **End Result**: You get a ChatPromptTemplate that's ready for action, crafted from your message lineup.

🏗️ **Modularity**: Like using building blocks, this method lets you construct prompts piece by piece for maximum flexibility.

In essence, chat prompt pipelining is about assembling conversations from logical blocks, creating a user-friendly and adaptable ChatPromptTemplate.


In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Setting the scene with a Cockney-themed system message
prompt = SystemMessage(content="Welcome to the East End Cockney Chat! 🇬🇧")

# Constructing a chat flow with dry humour
new_prompt = (
    prompt
    + HumanMessage(content="Alright, guv'nor?")
    + AIMessage(content="Not too shabby. Did you hear about the London fog?")
    + "{input}"
)

# Formatting the chat with the user's response
new_prompt.format_messages(input="No, what about it?")

print(new_prompt)


In [ ]:
new_prompt.messages

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.75)

output_parser = StrOutputParser()

chain = new_prompt | model | output_parser

# Running the chatbot to get the punchline
for chunk in chain.stream({"input":"No, what about it?"}):
  print(chunk, end="", flush=True)

# Prompt Composition


🧬 **Prompt Composition**: Reuse prompt segments with ease using the PipelinePrompt feature.

🏁 **1. Final Prompt**: The end product that you present to the model.

🔗 **2. Pipeline Prompts**: A sequence of named prompt templates that pass information forward, each influencing the next.

To summarize, PipelinePrompt allows for the efficient building of complex prompts by reusing and chaining together smaller, named components.


In [ ]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [ ]:
full_template = """{introduction}

{example}

{start}"""

full_prompt = PromptTemplate.from_template(full_template)

In [ ]:
full_prompt.input_variables

In [ ]:
introduction_template = """You are impersonating {person}."""

introduction_prompt = PromptTemplate.from_template(introduction_template)

In [ ]:
introduction_prompt.input_variables

In [ ]:
example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""

example_prompt = PromptTemplate.from_template(example_template)

In [ ]:
start_template = """Now, do this for real!

Q: {input}
A:"""

start_prompt = PromptTemplate.from_template(start_template)

In [ ]:
full_prompt

In [ ]:
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt)
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

In [ ]:
pipeline_prompt.input_variables

In [ ]:
last_prompt = pipeline_prompt.format(
    person="Elon Musk",
    example_q="What's your favorite car?",
    example_a="Tesla",
    input="What's your favorite social media site?"
)

print(last_prompt)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.75)

chain = pipeline_prompt | llm | StrOutputParser()

chain.invoke({
    "person":"Elon Musk","example_q":"What's your favorite car?",
    "example_a":"Tesla",
    "input":"What's your favorite social media site and why?"
    })

In [ ]:
from langchain.prompts import PromptTemplate, PipelinePromptTemplate

class CookingShowChatbot:
    def __init__(self):
        # Base template for the cooking show scenario
        self.full_template = """
        {introduction}

        {example_dish}

        {present_dish}"""
        self.full_prompt = PromptTemplate.from_template(self.full_template)

        # Introduction where the user impersonates a famous chef
        self.introduction_template = """Welcome to the cooking show! Today, you're channeling the spirit of Chef {chef_name}."""
        self.introduction_prompt = PromptTemplate.from_template(self.introduction_template)

        # Example dish made by the famous chef
        self.example_dish_template = """Remember when Chef {chef_name} made that delicious {example_dish_name}? It was a hit!"""
        self.example_dish_prompt = PromptTemplate.from_template(self.example_dish_template)

        # User's turn to present their dish
        self.present_dish_template = """Now, it's your turn! Show us how you make your {user_dish_name}. Let's get cooking!"""
        self.present_dish_prompt = PromptTemplate.from_template(self.present_dish_template)

        # Combining the prompts into a pipeline
        self.input_prompts = [
            ("introduction", self.introduction_prompt),
            ("example_dish", self.example_dish_prompt),
            ("present_dish", self.present_dish_prompt)
        ]
        self.pipeline_prompt = PipelinePromptTemplate(final_prompt=self.full_prompt,
                                                      pipeline_prompts=self.input_prompts
                                                      )

    def run_scenario(self, chef_name, example_dish_name, user_dish_name):
        chain = self.pipeline_prompt | llm | StrOutputParser()

        response = chain.invoke({"chef_name":chef_name, "example_dish_name":example_dish_name, "user_dish_name":user_dish_name})

        return response




In [ ]:
chatbot = CookingShowChatbot()
scenario = chatbot.run_scenario(chef_name="Gordon Ramsay", example_dish_name="Beef Wellington", user_dish_name="Vegan Lasagna")
print(scenario)